In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn import metrics
import xgboost as xg
from xgboost import XGBRegressor


def one_hot_encode(
    df: pd.DataFrame,
    column: str,
    prefix: str,
) -> pd.DataFrame:

    df_ = df.copy()
    df_encoded = pd.get_dummies(df_[column], dtype="int")
    for col in df_encoded:
        df_[f"{prefix}_{col}"] = df_encoded[col]

    return df_.drop(columns=[column])

In [13]:
vuelos = pd.read_parquet("../resources/flights_clean.parquet")
ventas = pd.read_parquet("../resources/sales_clean.parquet")

In [14]:
# Merge a ambos dataframes
vuelos_ventas = pd.merge(ventas, vuelos, on="Flight_ID")
vuelos_ventas.drop(columns=["index","Aeronave"], inplace=True)
vuelos_ventas

,Flight_ID,ProductType,ProductName,Quantity,TotalSales,Aeronave,DepartureStation,ArrivalStation,Destination_Type,Origin_Type,STD,STA,Capacity,Passengers,Bookings,Semana,Mes
0,a05290288259526edd3601160b10e1de,Botanas,Carne Seca Habanero,1,55.0,XA-VIQ,AW,AO,Ciudad Principal,Ciudad Principal,2023-08-28 19:20:00,2023-08-28 20:45:00,186,131.0,100.0,35,8
1,a05290288259526edd3601160b10e1de,Botanas,Cheetos,2,98.0,XA-VIQ,AW,AO,Ciudad Principal,Ciudad Principal,2023-08-28 19:20:00,2023-08-28 20:45:00,186,131.0,100.0,35,8
2,a05290288259526edd3601160b10e1de,Botanas,Ruffles Queso,6,294.0,XA-VIQ,AW,AO,Ciudad Principal,Ciudad Principal,2023-08-28 19:20:00,2023-08-28 20:45:00,186,131.0,100.0,35,8
3,a05290288259526edd3601160b10e1de,Refrescos,Coca Sin Azucar,2,96.0,XA-VIQ,AW,AO,Ciudad Principal,Ciudad Principal,2023-08-28 19:20:00,2023-08-28 20:45:00,186,131.0,100.0,35,8
4,a05290288259526edd3601160b10e1de,Licores,Jack And Coke,1,72.0,XA-VIQ,AW,AO,Ciudad Principal,Ciudad Principal,2023-08-28 19:20:00,2023-08-28 20:45:00,186,131.0,100.0,35,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2095947,6113fc0448cd548cd7e7d3f76e7872c5,Perecederos,Cuerno Clasico De Pavo,5,575.0,XA-VBB,AF,AW,Ciudad Principal,Ciudad Fronteriza,2023-11-02 23:10:00,2023-11-03 00:50:00,240,154.0,119.0,44,11
2095948,78b84bc241b924d6c89e171a34b0f60c,Perecederos,Cuerno Clasico De Pavo,1,115.0,XA-VXB,AT,BP,MX Amigos y Familia,Ciudad Principal,2023-10-05 06:00:00,2023-10-05 07:05:00,240,188.0,150.0,40,10
2095949,2d7d30a16334ee2db84e5bc027a4a186,Licores,Vino Tinto Sangre De Toro,1,155.0,XA-VAC,AO,BD,Playa,Ciudad Principal,2023-05-16 18:45:00,2023-05-16 19:35:00,180,149.0,112.0,20,5
2095950,7a56b895719a91ca6a98004cff956b7d,Licores,Vino Tinto Sangre De Toro,1,155.0,XA-VBT,AL,AW,Ciudad Principal,MX Amigos y Familia,2023-02-15 07:00:00,2023-02-15 08:25:00,220,210.0,170.0,7,2


In [15]:
# Initialize LabelEncoder
# Fit label encoder and transform the data

vuelos_ventas = vuelos_ventas.groupby("Flight_ID").agg(
    {
        "TotalSales": "sum",
        "Bookings": "mean",
        "Passengers": "mean",
        "Origin_Type": "first",
        "Destination_Type": "first",
        "STD": "first",
        "STA": "first",
        "DepartureStation": "first",
        "ArrivalStation": "first",
        "Semana": "first",
        "Mes": "first",
        "Capacity": "first",
    }
)
vuelos_ventas["STD"] = pd.to_datetime(vuelos_ventas["STD"])
vuelos_ventas["STA"] = pd.to_datetime(vuelos_ventas["STA"])
vuelos_ventas["Duracion"] = vuelos_ventas["STA"] - vuelos_ventas["STD"]
vuelos_ventas["Duracion"] = vuelos_ventas["Duracion"].dt.total_seconds() / 60
vuelos_ventas["Hora"] = vuelos_ventas["STD"].dt.hour
vuelos_ventas["Dia"] = vuelos_ventas["STD"].dt.day
# Cambiar tipos de destinos a one-hot encoding
destination_type = vuelos_ventas["Destination_Type"].map(
    {
        "Ciudad Principal": "destino_ciudadprincipal",
        "Playa": "destino_playa",
        "MX Amigos y Familia": "destino_amigosfamilia",
        "Ciudad Fronteriza": "destino_ciudadfronteriza",
        "Ecoturismo": "destino_ecoturismo",
    }
)
destination_type = pd.get_dummies(destination_type, dtype="int")
vuelos_ventas = (
    pd.concat([vuelos_ventas, destination_type], axis=1)
    if "Destination_Type" in vuelos_ventas.columns
    else vuelos_ventas
)
vuelos_ventas = (
    vuelos_ventas.drop(columns=["Destination_Type"])
    if "Destination_Type" in vuelos_ventas.columns
    else vuelos_ventas
)
origen_type = vuelos_ventas["Origin_Type"].map(
    {
        "Ciudad Principal": "origen_ciudadprincipal",
        "Playa": "origen_playa",
        "MX Amigos y Familia": "origen_amigosfamilia",
        "Ciudad Fronteriza": "origen_ciudadfronteriza",
        "Ecoturismo": "origen_ecoturismo",
    }
)
origen_type = pd.get_dummies(origen_type, dtype="int")
vuelos_ventas = (
    pd.concat([vuelos_ventas, origen_type], axis=1)
    if "Origin_Type" in vuelos_ventas.columns
    else vuelos_ventas
)
vuelos_ventas = (
    vuelos_ventas.drop(columns=["Origin_Type"])
    if "Origin_Type" in vuelos_ventas.columns
    else vuelos_ventas
)
vuelos_ventas["Book_Pass"] = vuelos_ventas["Bookings"] / vuelos_ventas["Passengers"]
vuelos_ventas.drop(columns=["STD", "STA"], inplace=True)
vuelos_ventas = one_hot_encode(vuelos_ventas, "DepartureStation", "departure")
vuelos_ventas = one_hot_encode(vuelos_ventas, "ArrivalStation", "arrival")

,TotalSales,Bookings,Passengers,Semana,Mes,Capacity,Duracion,Hora,Dia,destino_amigosfamilia,...,arrival_BJ,arrival_BK,arrival_BL,arrival_BM,arrival_BN,arrival_BO,arrival_BP,arrival_BQ,arrival_BS,arrival_BT
Flight_ID,,,,,,,,,,,,,,,,,,,,,
00004a718edba9d9ef878d08f02ae057,2152.0,106.0,174.0,51,12,180,135.0,16,18,0,...,0,0,0,0,0,0,0,0,0,0
0000cd79c0c3a9c309df6064dcacaeea,2111.0,93.0,188.0,40,10,186,95.0,18,7,1,...,0,0,0,0,0,0,0,0,0,0
000163f0df9cbfc35c4c06645ec512f6,417.0,138.0,173.0,43,10,220,85.0,6,29,1,...,0,0,0,0,0,0,0,0,0,0
00017be73003a570dd426b155762769c,7561.0,95.0,156.0,42,10,240,225.0,9,17,0,...,0,0,0,0,0,0,0,0,0,0
0001a43836c338f8d8650aefb11672c9,7640.0,78.0,157.0,52,12,240,55.0,16,25,0,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
fffe47f84ecc55da94b2907a7317dd12,4919.0,107.0,203.0,23,6,240,135.0,15,7,0,...,0,0,0,1,0,0,0,0,0,0
fffe4a9c19ae2320e76f02939fdff957,3445.0,160.0,232.0,29,7,220,105.0,16,22,0,...,0,0,0,0,0,0,0,0,0,0
ffff138536f249f69340e0a8336f94a4,4358.0,101.0,246.0,23,6,240,200.0,14,11,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
# Hacemos el modelo de regesion
X = vuelos_ventas.drop(columns=["TotalSales"])
y = vuelos_ventas["TotalSales"]

In [19]:
x_train, x_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.3,
    random_state=0,
)

eval_set = [(x_test, y_test)]

train_dmatrix = xg.DMatrix(data=x_train, label=y_train)
test_dmatrix = xg.DMatrix(data=x_test, label=y_test)

# xgb_r = XGBRegressor(n_estimators=10, seed=123)
xgb_r = XGBRegressor(learning_rate=0.3, n_estimators=100)
xgb_r.fit(x_train, y_train, eval_set=eval_set, verbose=True)
y_pred = xgb_r.predict(x_test)

[0]	validation_0-rmse:2481.74894
[1]	validation_0-rmse:2198.97013
[2]	validation_0-rmse:2033.32798
[3]	validation_0-rmse:1907.75976
[4]	validation_0-rmse:1834.64862
[5]	validation_0-rmse:1773.85720
[6]	validation_0-rmse:1717.03975
[7]	validation_0-rmse:1690.80258
[8]	validation_0-rmse:1672.18555
[9]	validation_0-rmse:1654.92034
[10]	validation_0-rmse:1639.77394
[11]	validation_0-rmse:1630.80127
[12]	validation_0-rmse:1618.72050
[13]	validation_0-rmse:1610.92381
[14]	validation_0-rmse:1605.80254
[15]	validation_0-rmse:1602.88202
[16]	validation_0-rmse:1596.54765
[17]	validation_0-rmse:1590.12355
[18]	validation_0-rmse:1583.64605
[19]	validation_0-rmse:1581.56075
[20]	validation_0-rmse:1579.30407
[21]	validation_0-rmse:1577.54724
[22]	validation_0-rmse:1576.87235
[23]	validation_0-rmse:1571.83939
[24]	validation_0-rmse:1567.25605
[25]	validation_0-rmse:1565.76184
[26]	validation_0-rmse:1564.85253
[27]	validation_0-rmse:1562.18313
[28]	validation_0-rmse:1561.84044
[29]	validation_0-rmse:1

In [20]:
# Evaluamos el modelo
print("R2 Score:", metrics.r2_score(y_test, y_pred))
print("Mean Absolute Error:", metrics.mean_absolute_error(y_test, y_pred))
print("Mean Squared Error:", metrics.mean_squared_error(y_test, y_pred))
print("Root Mean Squared Error:", np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

R2 Score: 0.7197184432698949
Mean Absolute Error: 967.466745998056
Mean Squared Error: 2333468.8749034493
Root Mean Squared Error: 1527.569597400868


c:\Users\Franco Mendoza\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Users\Franco Mendoza\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
c:\Users\Franco Mendoza\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Users\Franco Mendoza\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is